In [12]:
import os

DATA_DIR = "../data"
JSON_FILE = os.path.join(DATA_DIR, "books.json")
DB_FILE = os.path.join(DATA_DIR, "library.db")

print(f"Target Database: {DB_FILE}")

Target Database: ../data\library.db


In [13]:
import json

with open(JSON_FILE, 'r') as f:
    raw_data = json.load(f)

cleaned_books = []

for item in raw_data:
    raw_author = item.get("Author", "")
    
    if "," in raw_author:
        last_name, first_name = raw_author.split(",", 1)
        author_last_name = last_name.strip()
        author_first_name = first_name.strip()
    else:
        author_last_name = raw_author.strip()
        author_first_name = ""

    book = {
        "book_title": item.get("Title", "Unknown Title").strip(),
        "author_first_name": author_first_name,
        "author_last_name": author_last_name,
        "genre": item.get("Genre", "Uncategorized").strip(),
        "sub_genre": item.get("SubGenre", "General").strip(),
    }

    cleaned_books.append(book)


print(f"Processed {len(cleaned_books)} books.")
for book in cleaned_books[:3]:
    print(book)


Processed 211 books.
{'book_title': 'Fundamentals of Wavelets', 'author_first_name': 'Jaideva', 'author_last_name': 'Goswami', 'genre': 'tech', 'sub_genre': 'signal_processing'}
{'book_title': 'Data Smart', 'author_first_name': 'John', 'author_last_name': 'Foreman', 'genre': 'tech', 'sub_genre': 'data_science'}
{'book_title': 'God Created the Integers', 'author_first_name': 'Stephen', 'author_last_name': 'Hawking', 'genre': 'tech', 'sub_genre': 'mathematics'}


In [14]:
import sqlite3

conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

In [15]:
create_table_query = """
CREATE TABLE IF NOT EXISTS books(
    id TEXT PRIMARY KEY,
    book_title TEXT NOT NULL,
    author_first_name TEXT,
    author_last_name TEXT,
    genre TEXT,
    sub_genre TEXT);
"""

cursor.execute(create_table_query)
conn.commit()

print("Database and Table 'books' created successfully.")

Database and Table 'books' created successfully.


In [16]:
for index, book in enumerate(cleaned_books, start=1):
    book_id = f"BO{index}"

    cursor.execute("""
        INSERT INTO books (
            id,
            book_title,
            author_first_name,
            author_last_name,
            genre,
            sub_genre
        ) VALUES (?, ?, ?, ?, ?, ?)
    """, (
        book_id,
        book["book_title"],
        book["author_first_name"],
        book["author_last_name"],
        book["genre"],
        book["sub_genre"]
    ))

conn.commit()

In [17]:
cursor.execute("""SELECT id, book_title FROM books WHERE LOWER(author_last_name) =LOWER(?);""",("Hawking",))
print(cursor.fetchall())

[('BO3', 'God Created the Integers'), ('BO94', 'Theory of Everything, The')]


In [ ]:
def get_random_books_by_subgenre(sub_genre):
    query = """
    SELECT book_title, author_first_name, author_last_name, sub_genre 
    FROM books 
    WHERE sub_genre = ? 
    ORDER BY RANDOM() 
    LIMIT 3;
    """
    cursor.execute(query, (sub_genre,))
    results = cursor.fetchall()
    
    if not results:
        print("No books found for this sub-genre.")
        return

    for row in results:
        title, first, last, sub = row
        print(f"Found: '{title}' by {first} {last} ({sub})")

get_random_books_by_subgenre('data_science')

Found: 'Pattern Classification' by Hart Duda (data_science)
Found: 'Signal and the Noise, The' by Nate Silver (data_science)
Found: 'Data Mining Handbook' by Robert Nisbet (data_science)


In [ ]:
conn.close()